In [1]:
library(INLA)
library(rSPDE)
library(readr)
library(fmesher)
library(inlabru)
data_ori <- as.data.frame(read_csv("tmp/1180_vol.csv"))
logpt<-log(data_ori$Close)
logreturn <-logpt[2:length(logpt)]-logpt[1:(length(logpt)-1)]
date <- data_ori$Date[1:(length(data_ori$Date)-1)]
   
date_timeindex<-as.numeric(as.POSIXct(date)) 
date_timeindex <- (date_timeindex - min(date_timeindex))/86400
mesh <- fm_mesh_1d(date_timeindex)
data = list(logreturn = logreturn, times = date_timeindex)





Loading required package: Matrix



Loading required package: sp

This is INLA_24.04.01 built 2024-04-01 19:43:22 UTC.
 - See www.r-inla.org/contact-us for how to get help.
 - List available models/likelihoods/etc with inla.list.models()
 - Use inla.doc(<NAME>) to access documentation

This is rSPDE 2.3.3.9000
- See https://davidbolin.github.io/rSPDE for vignettes and manuals.

Rows: 2372 Columns: 10
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
dbl  (9): Open, High, Low, Close, Volume, Dividends, Stock Splits, vol10, vol30
date (1): Date

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [2]:
remotes::install_github("inlabru-org/inlabru", ref = "devel")

Skipping install of 'inlabru' from a github remote, the SHA1 (81a6cb7a) has not changed since last install.
  Use `force = TRUE` to force installation



In [3]:
packageVersion("inlabru")

[1] ‘2.10.1.9002’

In [4]:
build_index <- function(data_size, window) {
    train <- test <- list()

    k = 1
    for (i in seq(from = window, to = data_size - window, by = window)) {
        train[[k]] <- 1:i
        test[[k]] <- (i + 1):(i + window)
        k = k + 1
    }
    index = list(train = train, test = test)
    return(index)
}


In [5]:

library(glue)
config_id<-7
run_id=7
fit_path="/home/jianj0c/project/STP/notebook_SPDE/stochasticVolatility/Log/7.fitting/fits"
results<-readRDS(file.path(fit_path, glue("fit_{config_id}.rds")))
rough_stochvol<-results$fit[[run_id+1]]$fit

config_id<-21
run_id=3
fit_path="/home/jianj0c/project/STP/notebook_SPDE/stochasticVolatility/Log/7.fitting/fits"
results<-readRDS(file.path(fit_path, glue("fit_{config_id}.rds")))
rough_stochvol.t<-results$fit[[run_id+1]]$fit

config_id<-10
run_id=9
fit_path="/home/jianj0c/project/STP/notebook_SPDE/stochasticVolatility/Log/7.fitting/fits"
results<-readRDS(file.path(fit_path, glue("fit_{config_id}.rds")))
rough_stochvol.nig<-results$fit[[run_id+1]]$fit



config_id<-33
run_id=6
fit_path="/home/jianj0c/project/STP/notebook_SPDE/stochasticVolatility/Log/7.fitting/fits"
results<-readRDS(file.path(fit_path, glue("fit_{config_id}.rds")))
rough_stochvolln<-results$fit[[run_id+1]]$fit

In [6]:
summary(rough_stochvol)

Warning message in bru_info_upgrade(object[["bru_info"]], old = old, new_version = new_version):
“Old bru_info object version 2.10.1 detected. Attempting upgrade to version 2.10.1.9002”
Detected bru_info version 2.10.1

Upgraded bru_info to 2.10.1.9002



inlabru version: 2.10.1.9002
INLA version: 24.04.01
Components:
f: main = cgeneric(times), group = exchangeable(1L), replicate = iid(1L)
Intercept: main = linear(1), group = exchangeable(1L), replicate = iid(1L)
Likelihoods:
  Family: 'stochvol'
    Data class: 'list'
    Predictor: logreturn ~ .
Time used:
    Pre = 0.12, Running = 53, Post = 0.28, Total = 53.4 
Fixed effects:
             mean     sd 0.025quant 0.5quant 0.975quant    mode kld
Intercept -49.231 14.613    -77.877  -49.234    -20.567 -49.234   0

Random effects:
  Name	  Model
    f CGeneric

Model hyperparameters:
              mean    sd 0.025quant 0.5quant 0.975quant  mode
Theta1 for f -4.47 0.029      -4.52    -4.47      -4.41 -4.47
Theta2 for f -4.55 0.057      -4.66    -4.55      -4.44 -4.55
Theta3 for f -5.15 0.045      -5.24    -5.15      -5.06 -5.15

Deviance Information Criterion (DIC) ...............: -142446.55
Deviance Information Criterion (DIC, saturated) ....: -3.64e+37
Effective number of parameters ...

In [7]:
models=list(rough_stochvol,rough_stochvol.t,rough_stochvol.nig,rough_stochvolln)

In [8]:
res2 <- cross_validation(models,train_test_indexes = build_index(length(date_timeindex), 252))

Fold: 1 / 8 
Model: Model 1 
Generating samples...


ERROR: Error in (function (..., row.names = NULL, check.rows = FALSE, check.names = TRUE, : arguments imply differing number of rows: 252, 0
